In [314]:
#####Setting Up

import numpy as np
import pandas as pd
import scipy as sp

#####Importing data
df = pd.read_csv("C:/Users/dariu/Desktop/Kaggle/UFC/ufc-master-16-09-20.csv")
df_upcoming = pd.read_csv("C:/Users/dariu/Desktop/Kaggle/UFC/ufc_fn_covington_woodley.csv")

####Combining df
df_combined = df_upcoming.append(df)

In [315]:
### Shortcut vs tutorial to make winner a numeric value
df_combined["Winner"]=df_combined["Winner"].apply(lambda x: 1 if x == "Blue" else 0)

### Making sure winner is numeric
df_combined["Winner"] = pd.to_numeric(df_combined["Winner"], errors = "coerce")

### Making sure date is in date.time  (tutorial made a mistake here, setting second df_combined as only df)
df_combined["date"]= pd.to_datetime(df_combined["date"])

###Splitting train and test set
num_upcoming = len(df_upcoming)
df_train=df_combined[num_upcoming:]
df_test=df_combined[:num_upcoming]

### Fighters set up
fighters_test = df_test[["R_fighter","B_fighter"]]
odds_test= df_test[["R_odds","B_odds"]]

###Setting up model and classifier 
from sklearn.tree import DecisionTreeClassifier
my_model= DecisionTreeClassifier(max_depth =10)
my_features = ["win_streak_dif", "height_dif", "reach_dif","sig_str_dif", "avg_td_dif"]

In [316]:
#### Making dataframes with only the relevant features
df_train_prepped = df_train[my_features].copy()
df_test_prepped = df_test[my_features].copy()

####Making dummified datasets
df_train_prepped = pd.get_dummies(df_train_prepped)
df_test_prepped= pd.get_dummies(df_train_prepped)

### Aligining the dummified data sets (removed because it broke the system)
##### df_train_prepped, df_test_prepped = df_train_prepped.align(df_test_prepped, join="left", axis=1)

### Fill NA
df_test_prepped=df_test_prepped.fillna(0)

fighters_test_prepped = fighters_test[fighters_test.index.isin(df_test_prepped.index)]
odds_test_prepped = odds_test[odds_test.index.isin(df_test_prepped.index)]
label_df = df_train["Winner"]
my_model.fit(df_train_prepped, label_df)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [317]:
probs = my_model.predict_proba(df_test_prepped)
df_test_prepped

,win_streak_dif,height_dif,reach_dif,sig_str_dif,avg_td_dif
0,0,0.00,5.08,1.980000,0.750000
1,2,10.16,15.24,0.380000,1.840000
2,-1,-5.08,-15.24,0.760000,0.190000
3,0,10.16,5.08,6.920000,-1.630000
4,0,2.54,0.00,0.580000,0.230000
...,...,...,...,...,...
4350,0,0.00,2.54,-13.666667,0.000000
4351,-3,7.62,7.62,-18.000000,-4.666667
4352,0,-2.54,-2.12,-4.000000,1.000000
4353,0,10.16,7.62,-40.500000,-3.500000


In [318]:
probs

array([[0.57331378, 0.42668622],
       [0.55      , 0.45      ],
       [0.51515152, 0.48484848],
       ...,
       [0.59174964, 0.40825036],
       [0.69565217, 0.30434783],
       [0.51515152, 0.48484848]])

In [319]:
###Creating arrays of fights and odds
fighters_array= fighters_test_prepped.to_numpy()
odds_array = odds_test_prepped.to_numpy()
fighters_array
odds_array


array([[-360,  275],
       [ 130, -159],
       [-500,  360],
       [-137,  110],
       [-190,  150],
       [-240,  188],
       [-480,  350],
       [-250,  195],
       [-118, -106],
       [-200,  160],
       [-137,  110],
       [-190,  150],
       [ 210, -275],
       [-155,  125]], dtype=int64)

In [320]:
###Putting together fighs, odds and calculated odds
prob_list = np.array(list(zip(fighters_array, odds_array, probs)))
prob_list


array([[['Colby Covington', 'Tyron Woodley'],
        [-360, 275],
        [0.5733137829912024, 0.42668621700879766]],

       [['Donald Cerrone', 'Niko Price'],
        [130, -159],
        [0.55, 0.45]],

       [['Khamzat Chimaev', 'Gerald Meerschaert'],
        [-500, 360],
        [0.5151515151515151, 0.48484848484848486]],

       [['Johnny Walker', 'Ryan Spann'],
        [-137, 110],
        [1.0, 0.0]],

       [['Mackenzie Dern', 'Randa Markos'],
        [-190, 150],
        [0.5733137829912024, 0.42668621700879766]],

       [['Kevin Holland', 'Darren Stewart'],
        [-240, 188],
        [0.6956521739130435, 0.30434782608695654]],

       [['Mirsad Bektic', 'Eduardo Garagorri'],
        [-480, 350],
        [0.5917496443812233, 0.40825035561877665]],

       [['Mayra Bueno Silva', 'Mara Romero Borella'],
        [-250, 195],
        [1.0, 0.0]],

       [['Jordan Espinosa', 'David Dvorak'],
        [-118, -106],
        [0.6068376068376068, 0.39316239316239315]],

       [

In [321]:
### Setting up an odds calculator function
def get_bet_ev(odds, prob):
    if odds>0:
        return ((odds * prob)-(100*(1-prob)))
    else:
        return ((100/abs(odds))*100*prob - (100*(1-prob)))

In [326]:
#### 
for p in prob_list:
    red_ev = get_bet_ev(p[1][0], p[2][0])
    blue_ev = get_bet_ev(p[1][1], p[2][1])
    
    print(p[0][0], "(RED) vs ", p[0][1], "(BLUE)")
    print(p[0][0], "has a", "%.2f" % (p[2][0]*100), "% chance of winning. His odds are", p[1][0], "This gives him a single bet EV of", "%.2f" %red_ev)
    print(p[0][1], "has a", "%.2f" % (p[2][1]*100), "% chance of winning. His odds are", p[1][1], "This gives him a single bet EV of", "%.2f" %blue_ev)
    if red_ev >0:
        print("RED is a good bet")
    elif(blue_ev > 0):
        print("BLUE is a good bet")
    else:
        print ("There is NO good bet")
    print()

Colby Covington (RED) vs  Tyron Woodley (BLUE)
Colby Covington has a 57.33 % chance of winning. His odds are -360 This gives him a single bet EV of -26.74
Tyron Woodley has a 42.67 % chance of winning. His odds are 275 This gives him a single bet EV of 60.01
BLUE is a good bet

Donald Cerrone (RED) vs  Niko Price (BLUE)
Donald Cerrone has a 55.00 % chance of winning. His odds are 130 This gives him a single bet EV of 26.50
Niko Price has a 45.00 % chance of winning. His odds are -159 This gives him a single bet EV of -26.70
RED is a good bet

Khamzat Chimaev (RED) vs  Gerald Meerschaert (BLUE)
Khamzat Chimaev has a 51.52 % chance of winning. His odds are -500 This gives him a single bet EV of -38.18
Gerald Meerschaert has a 48.48 % chance of winning. His odds are 360 This gives him a single bet EV of 123.03
BLUE is a good bet

Johnny Walker (RED) vs  Ryan Spann (BLUE)
Johnny Walker has a 100.00 % chance of winning. His odds are -137 This gives him a single bet EV of 72.99
Ryan Spann ha